# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [48]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [49]:
nhl_df['team']=nhl_df['team'].str.replace(r'\*$','',regex=True)
nhl_df

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Los Angeles Kings,82,46,28,8,100,.610,206,174,0.40,0.01,.549,38,2014,NHL
167,Phoenix Coyotes,82,37,30,15,89,.543,216,231,-0.16,0.03,.457,31,2014,NHL
168,Vancouver Canucks,82,36,35,11,83,.506,196,223,-0.30,0.03,.451,31,2014,NHL
169,Calgary Flames,82,35,40,7,77,.470,209,241,-0.35,0.04,.402,28,2014,NHL


In [55]:
nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nhl_df['team']=nhl_df['team'].str.replace(r'\*$','',regex=True)
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NHL']=cities['NHL'].str.replace(r'\[note\s\d+\]$','',regex=True)
cities=cities.set_index('Metropolitan area')
citiesNHL=cities['NHL']
citiesNHL.replace(['','—'],np.nan,inplace=True)
citiesNHL.dropna(inplace=True)
citiesNHL_Aux=citiesNHL.copy()
citiesNHL_Aux.drop(['Detroit','Toronto','Columbus'], inplace=True)
l=citiesNHL_Aux.index.to_list()
l1=[]
l2=[]

for area in l:
    for team in citiesNHL_Aux[area].split():
        l1.append(area)
        l2.append(team)

for other in ['Detroit','Toronto','Columbus']:
        l2.append(citiesNHL[other])
        l1.append(other)

citiesNHL_Aux2=pd.Series(l2,index=l1)

for i in range(len(nhl_df)):
     if nhl_df.loc[i,'team'] in ['Detroit Red Wings', 'Toronto Maple Leafs','Columbus Blue Jackets']:
        nhl_df.loc[i,'team']=nhl_df.loc[i,'team'].split()[-2] + ' '+ nhl_df.loc[i,'team'].split()[-1]
     else:
        nhl_df.loc[i,'team']=nhl_df.loc[i,'team'].split()[-1]

nhl_df


,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Kings,82,46,28,8,100,.610,206,174,0.40,0.01,.549,38,2014,NHL
167,Coyotes,82,37,30,15,89,.543,216,231,-0.16,0.03,.457,31,2014,NHL
168,Canucks,82,36,35,11,83,.506,196,223,-0.30,0.03,.451,31,2014,NHL
169,Flames,82,35,40,7,77,.470,209,241,-0.35,0.04,.402,28,2014,NHL


In [60]:
list(nhl_df['team'])

['Division',
 'Lightning',
 'Bruins',
 'Maple Leafs',
 'Panthers',
 'Red Wings',
 'Canadiens',
 'Senators',
 'Sabres',
 'Division',
 'Capitals',
 'Penguins',
 'Flyers',
 'Blue Jackets',
 'Devils',
 'Hurricanes',
 'Islanders',
 'Rangers',
 'Division',
 'Predators',
 'Jets',
 'Wild',
 'Avalanche',
 'Blues',
 'Stars',
 'Blackhawks',
 'Division',
 'Knights',
 'Ducks',
 'Sharks',
 'Kings',
 'Flames',
 'Oilers',
 'Canucks',
 'Coyotes',
 'Division',
 'Canadiens',
 'Senators',
 'Bruins',
 'Maple Leafs',
 'Lightning',
 'Panthers',
 'Red Wings',
 'Sabres',
 'Division',
 'Capitals',
 'Penguins',
 'Blue Jackets',
 'Rangers',
 'Islanders',
 'Flyers',
 'Hurricanes',
 'Devils',
 'Division',
 'Blackhawks',
 'Wild',
 'Blues',
 'Predators',
 'Jets',
 'Stars',
 'Avalanche',
 'Division',
 'Ducks',
 'Oilers',
 'Sharks',
 'Flames',
 'Kings',
 'Coyotes',
 'Canucks',
 'Division',
 'Panthers',
 'Lightning',
 'Red Wings',
 'Bruins',
 'Senators',
 'Canadiens',
 'Sabres',
 'Maple Leafs',
 'Division',
 'Capitals',

In [59]:
'Devils' in nhl_df['team']

False

In [ ]:
['Red Wings','Maple Leafs','Blue Jackets']

In [30]:
citiesNHL_Aux2

New York City                  Rangers
New York City                Islanders
New York City                   Devils
Los Angeles                      Kings
Los Angeles                      Ducks
San Francisco Bay Area          Sharks
Chicago                     Blackhawks
Dallas–Fort Worth                Stars
Washington, D.C.              Capitals
Philadelphia                    Flyers
Boston                          Bruins
Minneapolis–Saint Paul            Wild
Denver                       Avalanche
Miami–Fort Lauderdale         Panthers
Phoenix                        Coyotes
Tampa Bay Area               Lightning
Pittsburgh                    Penguins
St. Louis                        Blues
Nashville                    Predators
Buffalo                         Sabres
Montreal                     Canadiens
Vancouver                      Canucks
Calgary                         Flames
Ottawa                        Senators
Edmonton                        Oilers
Winnipeg                 

In [31]:
len(citiesNHL_Aux2)

32

In [32]:
list(nhl_df['team'])

['Atlantic Division',
 'Tampa Bay Lightning',
 'Boston Bruins',
 'Toronto Maple Leafs',
 'Florida Panthers',
 'Detroit Red Wings',
 'Montreal Canadiens',
 'Ottawa Senators',
 'Buffalo Sabres',
 'Metropolitan Division',
 'Washington Capitals',
 'Pittsburgh Penguins',
 'Philadelphia Flyers',
 'Columbus Blue Jackets',
 'New Jersey Devils',
 'Carolina Hurricanes',
 'New York Islanders',
 'New York Rangers',
 'Central Division',
 'Nashville Predators',
 'Winnipeg Jets',
 'Minnesota Wild',
 'Colorado Avalanche',
 'St. Louis Blues',
 'Dallas Stars',
 'Chicago Blackhawks',
 'Pacific Division',
 'Vegas Golden Knights',
 'Anaheim Ducks',
 'San Jose Sharks',
 'Los Angeles Kings',
 'Calgary Flames',
 'Edmonton Oilers',
 'Vancouver Canucks',
 'Arizona Coyotes',
 'Atlantic Division',
 'Montreal Canadiens',
 'Ottawa Senators',
 'Boston Bruins',
 'Toronto Maple Leafs',
 'Tampa Bay Lightning',
 'Florida Panthers',
 'Detroit Red Wings',
 'Buffalo Sabres',
 'Metropolitan Division',
 'Washington Capitals'

In [43]:
nhl_df.loc[0,'team']='A'

In [44]:
nhl_df

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,A,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Los Angeles Kings,82,46,28,8,100,.610,206,174,0.40,0.01,.549,38,2014,NHL
167,Phoenix Coyotes,82,37,30,15,89,.543,216,231,-0.16,0.03,.457,31,2014,NHL
168,Vancouver Canucks,82,36,35,11,83,.506,196,223,-0.30,0.03,.451,31,2014,NHL
169,Calgary Flames,82,35,40,7,77,.470,209,241,-0.35,0.04,.402,28,2014,NHL


In [50]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]